<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 30px">Nội dung</div>
    
[1. Đọc dữ liệu](#1) 
    
[2. Transform data](#2)
    
[3. Model](#3)
    
[3.1 Train](#3.1)
    
[3.2 Đánh giá](#3.2)
    
[4. Test](#4)
    
[5. Kết quả](#5)   
    
    

<a id="1"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">1. Đọc dữ liệu 📩📩</div>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import re
from transformers import AutoTokenizer ,BertForSequenceClassification, AdamW
from sklearn.metrics import classification_report

In [4]:
# read train data
train_data=pd.read_csv("new_train.csv")
# read test data
test_data=pd.read_csv("new_test.csv")

In [5]:
train_data.head()

,text,label
0,our deed reason earthquak may ah forgiv us,1
1,forest fire near la rong sask canada,1
2,all resid ask shelter place beg notifi off...,1
3,peopl receiv wildfir evacu order california,1
4,just got sent photo rubi alka smoke wildfi...,1


In [6]:
test_data.head()

,id,text
0,0,just happen terribl car crash
1,2,heard earthquak differ citi stay safe everyon
2,3,forest fire spot pond gees flee across st...
3,9,apocalyps light spokan wildfir
4,11,typhoon soudelor kill cha taiwan


In [7]:
train_data=train_data.values.tolist()
test_data=test_data.values.tolist()

<a id="2"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">2. Transform data 🔄️</div>



In [8]:
class Mydata(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return len(self.data)

In [9]:
train, valid = train_test_split(train_data, test_size = 0.24)

In [10]:
train=Mydata(train)
valid=Mydata(valid)

In [11]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [12]:
print("Model max_length: ", tokenizer.model_max_length)

Model max_length:  512


In [13]:
def collate_fn(data: list[tuple[str, int]]):
    texts = []
    labels = []
    for content, label in data:
        texts.append(content)
        labels.append(label)
    input_ids = tokenizer.batch_encode_plus(texts, padding = True, truncation = True)['input_ids']
    input_ids = torch.tensor(input_ids)
    labels = torch.tensor(labels)
    return input_ids, labels

In [14]:
train_dataloader = torch.utils.data.DataLoader(dataset = train, batch_size = 32, collate_fn = collate_fn, shuffle = True)
valid_dataloader = torch.utils.data.DataLoader(dataset = valid, batch_size = 32, collate_fn = collate_fn)

In [15]:
for input_ids, labels in train_dataloader:
    print(input_ids, labels)
    break

tensor([[ 101, 7367, 6299,  ...,    0,    0,    0],
        [ 101, 7861, 5017,  ...,    0,    0,    0],
        [ 101, 3857, 2983,  ...,    0,    0,    0],
        ...,
        [ 101, 2065, 2017,  ...,    0,    0,    0],
        [ 101, 2128, 4168,  ...,    0,    0,    0],
        [ 101, 1045, 4797,  ...,    0,    0,    0]]) tensor([1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 1, 0, 1, 0, 0, 0, 0])


In [16]:
torch.cuda.is_available()

True

<a id="3"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">
3. Model 🤖🤖
</div>



In [17]:
class MyBertForClassification(torch.nn.Module):
    def __init__(self, num_labels) -> None:
        super().__init__()
        self.bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
    def forward(self, input_ids, labels):
        outputs = self.bert(input_ids=input_ids, labels=labels)
        probs = torch.softmax(outputs['logits'], dim = -1)
        preds = torch.argmax(probs, dim = -1)
        outputs['preds'] = preds
        return outputs

In [18]:
model = MyBertForClassification(2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [19]:
# Chuyển mô hình lên GPU
model.cuda()


MyBertForClassification(
  (bert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_featu

<a id="3.1"></a>
<div style=" background-color:#dceff2 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 30px">3.1 Train model ✍️✍️</div>



In [20]:
LR = 4e-6
EPOCH = 4
LOG_STEP = 30

In [21]:
optimizer = AdamW(model.parameters(), lr=LR)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
print(f"Train Loader Step: {len(train_dataloader)}")
print(f"Valid Loader Step: {len(valid_dataloader)}")

Train Loader Step: 181
Valid Loader Step: 58


In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


accuracy = []
precision = []
recall = []
f1 = []
loss_list = []

for epoch in range(EPOCH):
    running_loss = 0.0
    print(f"Epoch {epoch + 1}:")
    model.train()  # Chuyển sang chế độ huấn luyện
    for i, (input_id, label) in enumerate(train_dataloader):
        input_ids = input_id
        labels = label

        input_ids = input_ids.cuda()
        labels = labels.cuda()

        optimizer.zero_grad()

        outputs = model(input_ids, labels)
        loss = outputs['loss']
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % LOG_STEP == LOG_STEP-1:
            print('[Epoch %d, Batch %d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / LOG_STEP))
            running_loss = 0.0

    model.eval()
    valid_loss = 0
    true_labels = []
    pred_labels = []

    with torch.no_grad():
        for step, (input_ids, labels) in enumerate(valid_dataloader):
            input_ids = input_ids.cuda()
            labels = labels.cuda()
            outputs = model(input_ids, labels)
            loss = outputs['loss']
            valid_loss += loss.item()


            true_labels.extend(labels.tolist())
            pred_labels.extend(outputs['preds'].tolist())

    accuracy.append(accuracy_score(true_labels, pred_labels))
    precision.append(precision_score(true_labels, pred_labels, average='macro'))
    recall.append(recall_score(true_labels, pred_labels, average='macro'))
    f1.append(f1_score(true_labels, pred_labels, average='macro'))
    loss_list.append((valid_loss / len(valid_dataloader)))
    print(f"Valid loss: {valid_loss / len(valid_dataloader)}")

# Lưu mô hình
torch.save(model.state_dict(), 'bert_classifier.pth')
# model.load_state_dict(torch.load('bert_classifier.pth'))
# model.eval()

Epoch 1:
[Epoch 1, Batch 30] loss: 0.663
[Epoch 1, Batch 60] loss: 0.658
[Epoch 1, Batch 90] loss: 0.621
[Epoch 1, Batch 120] loss: 0.589
[Epoch 1, Batch 150] loss: 0.542
[Epoch 1, Batch 180] loss: 0.533
Valid loss: 0.5089365901618168
Epoch 2:
[Epoch 2, Batch 30] loss: 0.533
[Epoch 2, Batch 60] loss: 0.472
[Epoch 2, Batch 90] loss: 0.525
[Epoch 2, Batch 120] loss: 0.504
[Epoch 2, Batch 150] loss: 0.483
[Epoch 2, Batch 180] loss: 0.463
Valid loss: 0.46589065317449896
Epoch 3:
[Epoch 3, Batch 30] loss: 0.457
[Epoch 3, Batch 60] loss: 0.422
[Epoch 3, Batch 90] loss: 0.458
[Epoch 3, Batch 120] loss: 0.455
[Epoch 3, Batch 150] loss: 0.462
[Epoch 3, Batch 180] loss: 0.444
Valid loss: 0.4795720762219922
Epoch 4:
[Epoch 4, Batch 30] loss: 0.421
[Epoch 4, Batch 60] loss: 0.403
[Epoch 4, Batch 90] loss: 0.413
[Epoch 4, Batch 120] loss: 0.422
[Epoch 4, Batch 150] loss: 0.398
[Epoch 4, Batch 180] loss: 0.399
Valid loss: 0.45381132564668


<a id="3.2"></a>
<div style=" background-color:#dceff2 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 30px">
3.2 Đánh giá mô hình 👍👍</div>



In [25]:
df_score = pd.DataFrame({'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1, 'Loss': loss_list }, index=
            ['Epoch 1','Epoch 2','Epoch 3', 'Epoch 4'])

In [26]:
df_score

,Accuracy,Precision,Recall,F1,Loss
Epoch 1,0.760394,0.757347,0.756812,0.757064,0.508937
Epoch 2,0.783370,0.787287,0.772716,0.775980,0.465891
Epoch 3,0.770241,0.802361,0.749053,0.751822,0.479572
Epoch 4,0.792123,0.791916,0.785078,0.787273,0.453811


Hiệu suất của mô hình được cải thiện qua mỗi epoch trong đó `Accuracy`,	`Precision`, `Recall`, `F1` tăng dần và `Loss` giảm dần

<a id="4"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">
4. Test 🔍🔍
</div>

In [27]:
def collate_fn(data: list[tuple[str, int]]):
    texts = []
    ids = []
    for id, content in data:
        texts.append(content)
        ids.append(id)
    input_ids = tokenizer.batch_encode_plus(texts, padding = True, truncation = True)['input_ids']
    input_ids = torch.tensor(input_ids)
    ids = torch.tensor(ids)
    return input_ids, ids

In [28]:
inputs,ids=collate_fn(test_data)

In [29]:
test=Mydata(test_data)

In [30]:
test_dataloader = torch.utils.data.DataLoader(dataset = test_data, batch_size = 1, collate_fn = collate_fn)

In [31]:
result_test={
    'id':[],
    'target':[]
}

In [32]:
with torch.no_grad():
    for step, (input_ids, idx) in enumerate(test_dataloader):
        input_ids = input_ids.cuda()
        # labels = labels.cuda()
        outputs = model(input_ids=input_ids, labels=None)
        preds = outputs['preds']
        span = preds[0].item()
        result_test['id'].append(idx.item())
        result_test['target'].append(span)

In [33]:
df=pd.DataFrame(result_test)

In [34]:
df

,id,target
0,0,1
1,2,0
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,0
3259,10865,0
3260,10868,1
3261,10874,1


In [35]:
df.to_csv("result1.csv",index=False)

<a id="5"></a>
<div style=" background-color:#cdf7c1 ; color:#e3395e; padding: 15px; border-radius:5px;text-align: center; border-style: solid; border-color: green; font-size: 35px">
5. Kết quả 📜📜
</div>



![result](result.png)  